Мне жизненно необходимо реализовать алгоритм Витерби для определения частей речи. Не знаю, выйдет ли что-нибудь, но я хотя бы попробую.

In [6]:
import pandas as pd
import numpy as np
from numpy import log as ln
import nltk
from nltk.probability import FreqDist
from nltk.corpus import treebank

In [7]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [30]:
possible_tags = set()
possible_words = set()
for word, tag in treebank.tagged_words():
  possible_words.add(word)
  possible_tags.add(tag)

In [38]:
tags_to_tags = pd.DataFrame(data=0, index=possible_tags, columns=possible_tags)
words_to_tags = pd.DataFrame(data=1, index=possible_words, columns=possible_tags)

In [39]:
for sentence in treebank.tagged_sents():
  for i_word, word_tag in enumerate(sentence):
    if i_word != 0:
      tags_to_tags.at[sentence[i_word - 1][1], word_tag[1]] += 1
    words_to_tags.at[word_tag[0], word_tag[1]] += 1

In [67]:
def scale(df):
  return ln(df/df.sum())

In [68]:
scaled_tags_to_tags = scale(tags_to_tags)
scaled_words_to_tags = scale(words_to_tags)

Часть про тэгирование предложения. И она не особо работает, увы.

In [230]:
sentence = "It is a nice green tree, I like it"
sentence = list(sentence.lower().split())

In [231]:
tagged_sentence = pd.DataFrame(data=0, index=possible_tags, columns=sentence)
tagged_sentence.insert(0, column='initial_value', value=ln(1/len(possible_tags)))

In [232]:
indexes_we_came_from = np.array([[0 for j in range(len(sentence))] for i in range(len(tagged_sentence.index))])

In [233]:
for i_col, col in enumerate(tagged_sentence.columns[1:]):
  for i_row, row in enumerate(tagged_sentence.index):

    mx = float("-inf")
    index_mx = 0

    for i_tag, tag in enumerate(tagged_sentence.index):
      try:
        word_value = scaled_words_to_tags.at[col, row]
      except KeyError:
        word_value = 0
      value = word_value + scaled_tags_to_tags.at[tag, row] + tagged_sentence.iat[i_tag, i_col]
      index = i_tag
      if value > mx:
        mx = value
        index_mx = index

    indexes_we_came_from[i_row][i_col] = index_mx
    tagged_sentence.at[row, col] = mx

In [234]:
start = tagged_sentence.iloc[:, -1].argmax()
array_of_indexes = [start]
for i in range(len(sentence)-1, 0, -1):
  array_of_indexes.append(indexes_we_came_from[start, i])
  start = indexes_we_came_from[start, i]
array_of_indexes.reverse()

In [235]:
tags = tagged_sentence.index[array_of_indexes]

In [242]:
final = list(zip(sentence, tags))

In [243]:
print(final)

[('it', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('nice', 'JJ'), ('green', 'JJ'), ('tree,', 'NN'), ('i', 'NN'), ('like', 'IN'), ('it', 'PRP')]


Ну и немножечко сравнений с нормальными тэгами.

In [228]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [239]:
print(nltk.pos_tag(sentence))

[('it', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('nice', 'JJ'), ('green', 'JJ'), ('tree,', 'NN'), ('i', 'NN'), ('like', 'IN'), ('it', 'PRP')]
